# Objetivo: 
Extracao de dados relacionados a captura, utilização e armazenamento de carbono (CCUS).

# Configurações iniciais

## Bibliotecas

In [0]:
!pip install openpyxl --quiet
!pip install unidecode --quiet


[notice] A new release of pip available: 22.2.2 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [0]:
import pandas as pd
import re
from pyspark.sql import functions as sf
from unidecode import unidecode

## Parâmetros

In [0]:
# Definições do arquivo Excel
sheetname = "CCUS Projects Database"
cellposition = "A1"

## Caminhos

In [0]:
# Caminho dos dados
path_arquivo = "https://github.com/samuelrubert/pucrio-mvp-engenhariadados/raw/main/Dados/IEA_CCUS_Projects_Database.xlsx"

# Extração

## Leitura do arquivo Excel

In [0]:
# Ler pandas pois o spark não consegue ler deste local
df_pandas = pd.read_excel(path_arquivo, sheet_name=sheetname, header=0)
df_pandas.head()

,Project name,ID,Country,Partners,Project type,Announcement,FID,Operation,Suspension/decommissioning,Project Status,Project phase,Announced capacity (Mt CO2/yr),Estimated capacity by IEA (Mt CO2/yr),Sector,Fate of carbon,Part of CCUS hub,Region,Ref 1,Ref 2,Ref 3,Ref 4,Ref 5,Ref 6,Ref 7,Link 1,Link 2,Link 3,Link 4,Link 5,Link 6,Link 7
0,3D DMX ArcelorMittal and IFPEN Dunkirk (full-s...,1,France,"ArcelorMittal, ifp, Axens, Uetikon, Grassco, b...",Capture,2019.0,NaN,2025.0,NaN,Planned,2.0,0.7,0.70,Iron and steel,Unknown/unspecified,NaN,Europe,https://automotive.arcelormittal.com/news_and_...,https://3d-ccus.com/3d-overview/,NaN,NaN,NaN,NaN,NaN,Link 1,Link 2,NaN,NaN,NaN,NaN,NaN
1,3D DMX ArcelorMittal and IFPEN Dunkirk 'REUZE',751,France,"ArcelorMittal, Engie, Infinium",CCU,2022.0,2024.0,2025.0,NaN,Planned,1.0,0.3,0.30,Iron and steel,Use,NaN,Europe,https://newsroom.engie.com/actualites/engie-et...,https://corporate.arcelormittal.com/climate-ac...,NaN,NaN,NaN,NaN,NaN,Link 1,Link 2,NaN,NaN,NaN,NaN,NaN
2,7 Blue Ammonia Facility,1055,Qatar,"QAFCO, thyssenkrupp Uhde/Consolidated Contract...",Full Chain,2022.0,2022.0,2026.0,NaN,Under construction,NaN,1.5,1.50,Hydrogen or ammonia,Unknown/unspecified,NaN,Middle East,https://www.qafco.qa/content/qafco-7-announcement,https://www.thyssenkrupp.com/en/newsroom/press...,NaN,NaN,NaN,NaN,NaN,Link 1,Link 2,NaN,NaN,NaN,NaN,NaN
3,8Rivers H2 (8RH2) (WY),3,United States,"8Rivers, Wyoming Energy Authority",Capture,2022.0,NaN,NaN,NaN,Planned,NaN,NaN,NaN,Hydrogen or ammonia,Dedicated storage,NaN,North America,https://8rivers.com/portfolio/8-rivers-hydrogen/#,NaN,NaN,NaN,NaN,NaN,NaN,Link 1,NaN,NaN,NaN,NaN,NaN,NaN
4,Abadi CCS/CCUS,227,Indonesia,"Inpex Masela 65%, Shell (trying to find a buye...",Full chain,2018.0,NaN,2027.0,NaN,Planned,NaN,2.41,2.41,Natural gas processing/LNG,Unknown/unspecified,NaN,Other Asia Pacific,https://www.upstreamonline.com/lng/inpex-furth...,https://www.ogj.com/pipelines-transportation/l...,https://www.spglobal.com/commodityinsights/en/...,NaN,NaN,NaN,NaN,Link 1,Link 2,Link 3,NaN,NaN,NaN,NaN


In [0]:
# Verificar os tipos de dados inferidos pelo pandas
df_pandas.dtypes

Project name                              object
ID                                         int64
Country                                   object
Partners                                  object
Project type                              object
Announcement                             float64
FID                                      float64
Operation                                float64
Suspension/decommissioning               float64
Project Status                            object
Project phase                            float64
Announced capacity (Mt CO2/yr)            object
Estimated capacity by IEA (Mt CO2/yr)    float64
Sector                                    object
Fate of carbon                            object
Part of CCUS hub                          object
Region                                    object
Ref 1                                     object
Ref 2                                     object
Ref 3                                     object
Ref 4               

## Conversão em dataframe Spark

In [0]:
df = spark.createDataFrame(df_pandas)
display(df.limit(10))

Project name,ID,Country,Partners,Project type,Announcement,FID,Operation,Suspension/decommissioning,Project Status,Project phase,Announced capacity (Mt CO2/yr),Estimated capacity by IEA (Mt CO2/yr),Sector,Fate of carbon,Part of CCUS hub,Region,Ref 1,Ref 2,Ref 3,Ref 4,Ref 5,Ref 6,Ref 7,Link 1,Link 2,Link 3,Link 4,Link 5,Link 6,Link 7
3D DMX ArcelorMittal and IFPEN Dunkirk (full-scale),1,France,"ArcelorMittal, ifp, Axens, Uetikon, Grassco, brevik, TotalEnergies (T&S)",Capture,2019.0,NaN,2025.0,NaN,Planned,2.0,0.7,0.7,Iron and steel,Unknown/unspecified,NaN,Europe,https://automotive.arcelormittal.com/news_and_stories/news/2019DMXproject,https://3d-ccus.com/3d-overview/,NaN,NaN,NaN,NaN,NaN,Link 1,Link 2,NaN,NaN,NaN,NaN,NaN
3D DMX ArcelorMittal and IFPEN Dunkirk 'REUZE',751,France,"ArcelorMittal, Engie, Infinium",CCU,2022.0,2024.0,2025.0,NaN,Planned,1.0,0.3,0.3,Iron and steel,Use,NaN,Europe,https://newsroom.engie.com/actualites/engie-et-infinium-annoncent-un-partenariat-pour-developper-un-hub-industriel-denvergure-europeenne-de-production-de-carburants-de-synthese-a-dunkerque-6d4d-ff316.html,https://corporate.arcelormittal.com/climate-action/decarbonisation-technologies/the-3d-project-dmx-demonstration-in-dunkirk,NaN,NaN,NaN,NaN,NaN,Link 1,Link 2,NaN,NaN,NaN,NaN,NaN
7 Blue Ammonia Facility,1055,Qatar,"QAFCO, thyssenkrupp Uhde/Consolidated Contractors Company (EPC)",Full Chain,2022.0,2022.0,2026.0,NaN,Under construction,NaN,1.5,1.5,Hydrogen or ammonia,Unknown/unspecified,NaN,Middle East,https://www.qafco.qa/content/qafco-7-announcement,https://www.thyssenkrupp.com/en/newsroom/press-releases/pressdetailpage/thyssenkrupp-uhde-to-build-world-scale-blue-ammonia-plant-in-qatar-138369,NaN,NaN,NaN,NaN,NaN,Link 1,Link 2,NaN,NaN,NaN,NaN,NaN
8Rivers H2 (8RH2) (WY),3,United States,"8Rivers, Wyoming Energy Authority",Capture,2022.0,NaN,NaN,NaN,Planned,NaN,NaN,NaN,Hydrogen or ammonia,Dedicated storage,NaN,North America,https://8rivers.com/portfolio/8-rivers-hydrogen/#,NaN,NaN,NaN,NaN,NaN,NaN,Link 1,NaN,NaN,NaN,NaN,NaN,NaN
Abadi CCS/CCUS,227,Indonesia,"Inpex Masela 65%, Shell (trying to find a buyer for its shares), ITB (feasibility)",Full chain,2018.0,NaN,2027.0,NaN,Planned,NaN,2.41,2.41,Natural gas processing/LNG,Unknown/unspecified,NaN,Other Asia Pacific,https://www.upstreamonline.com/lng/inpex-further-delays-flagship-abadi-lng-project/2-1-1173044,https://www.ogj.com/pipelines-transportation/lng/article/14292056/inpex-submits-ccs-plan-for-abadi-lng-project,https://www.spglobal.com/commodityinsights/en/market-insights/latest-news/lng/092823-interview-indonesia-to-finalize-presidential-ccsccus-regulation-by-year-end,NaN,NaN,NaN,NaN,Link 1,Link 2,Link 3,NaN,NaN,NaN,NaN
Ackerman Combined Cycle Plant (MS),975,United States,"Trifecta Renewable Solutions, Tennessee Valley Authority",Capture,2023.0,NaN,NaN,NaN,Planned,NaN,1.8,1.8,Power and heat,Dedicated storage,Red Hills CO2 storage hub (MS),North America,https://www.energy.gov/fecm/project-selections-foa-2711-carbon-storage-validation-and-testing-round-2,NaN,NaN,NaN,NaN,NaN,NaN,Link 1,NaN,NaN,NaN,NaN,NaN,NaN
Acorn CCS phase 1-Capture at St Fergus,7,United Kingdom,"Storegga(30%), Shell (30%), Harbour (30%), North Sea Midstream partners (10%)",Capture,2016.0,2024.0,2028.0,NaN,Planned,NaN,0.3,0.3,Natural gas processing/LNG,Dedicated storage,Acorn CCS,Europe,https://www.upstreamonline.com/energy-transition/five-proposed-uk-carbon-capture-projects-meet-governments-eligibility-test/2-1-1047222,https://www.offshore-energy.biz/carbon-clean-to-carry-out-feed-on-acorn-carbon-capture-plant/,https://www.backthescottishcluster.co.uk/the-scottish-cluster-supply-chain,https://www.storegga.earth/news/2021/news/acorn-project-partners-storegga-shell-u-k-and-harbour-energy-sign-mou-with-the-owners-of-the-segal-and-fuka-gas-terminals-at-st-fergus/,https://www.scottishconstructionnow.com/articles/acorn-project-partners-select-carbon-clean-to-deliver-st-fergus-carbon-capture-plant-feed-study,https://www.energyvoice.com/oi

In [0]:
# Este seria o bronze

# Transformação

## Tratar nomes das colunas
As colunas devem ser renomeadas para serem compatíveis com uma tabela em banco de dados.

In [0]:
# Dicionário de nomes
dict_rename = {}

# Criar nomes tratados
for c in df.columns:
    nome_ok = c.strip()
    nome_ok = nome_ok.replace('/', '_')
    nome_ok = re.sub('[^A-Za-z0-9_ ]+', '', unidecode(nome_ok))
    nome_ok = re.sub('\s+','_', nome_ok)
    dict_rename[c] = nome_ok

# Renomear colunas no dataframe
df = df.withColumnsRenamed(dict_rename)

display(df.limit(3))

Project_name,ID,Country,Partners,Project_type,Announcement,FID,Operation,Suspension_decommissioning,Project_Status,Project_phase,Announced_capacity_Mt_CO2_yr,Estimated_capacity_by_IEA_Mt_CO2_yr,Sector,Fate_of_carbon,Part_of_CCUS_hub,Region,Ref_1,Ref_2,Ref_3,Ref_4,Ref_5,Ref_6,Ref_7,Link_1,Link_2,Link_3,Link_4,Link_5,Link_6,Link_7
3D DMX ArcelorMittal and IFPEN Dunkirk (full-scale),1,France,"ArcelorMittal, ifp, Axens, Uetikon, Grassco, brevik, TotalEnergies (T&S)",Capture,2019.0,NaN,2025.0,NaN,Planned,2.0,0.7,0.7,Iron and steel,Unknown/unspecified,NaN,Europe,https://automotive.arcelormittal.com/news_and_stories/news/2019DMXproject,https://3d-ccus.com/3d-overview/,NaN,NaN,NaN,NaN,NaN,Link 1,Link 2,NaN,NaN,NaN,NaN,NaN
3D DMX ArcelorMittal and IFPEN Dunkirk 'REUZE',751,France,"ArcelorMittal, Engie, Infinium",CCU,2022.0,2024.0,2025.0,NaN,Planned,1.0,0.3,0.3,Iron and steel,Use,NaN,Europe,https://newsroom.engie.com/actualites/engie-et-infinium-annoncent-un-partenariat-pour-developper-un-hub-industriel-denvergure-europeenne-de-production-de-carburants-de-synthese-a-dunkerque-6d4d-ff316.html,https://corporate.arcelormittal.com/climate-action/decarbonisation-technologies/the-3d-project-dmx-demonstration-in-dunkirk,NaN,NaN,NaN,NaN,NaN,Link 1,Link 2,NaN,NaN,NaN,NaN,NaN
7 Blue Ammonia Facility,1055,Qatar,"QAFCO, thyssenkrupp Uhde/Consolidated Contractors Company (EPC)",Full Chain,2022.0,2022.0,2026.0,NaN,Under construction,NaN,1.5,1.5,Hydrogen or ammonia,Unknown/unspecified,NaN,Middle East,https://www.qafco.qa/content/qafco-7-announcement,https://www.thyssenkrupp.com/en/newsroom/press-releases/pressdetailpage/thyssenkrupp-uhde-to-build-world-scale-blue-ammonia-plant-in-qatar-138369,NaN,NaN,NaN,NaN,NaN,Link 1,Link 2,NaN,NaN,NaN,NaN,NaN


## Tratar dados

### Remover NaN
Os valores NaN nas colunas do tipo float são substituídos por nulos, pois eles interferem nas agregações de valores em certas ferramentas, como o Power BI.
Ademais, vamos remover os valores NaN nas colunas de texto, pois se trata de um valor sem significado para o público de negócio.

In [0]:
df = df.replace(float("nan"), None)
df = df.replace("NaN", None)

display(df.limit(3))

Project_name,ID,Country,Partners,Project_type,Announcement,FID,Operation,Suspension_decommissioning,Project_Status,Project_phase,Announced_capacity_Mt_CO2_yr,Estimated_capacity_by_IEA_Mt_CO2_yr,Sector,Fate_of_carbon,Part_of_CCUS_hub,Region,Ref_1,Ref_2,Ref_3,Ref_4,Ref_5,Ref_6,Ref_7,Link_1,Link_2,Link_3,Link_4,Link_5,Link_6,Link_7
3D DMX ArcelorMittal and IFPEN Dunkirk (full-scale),1,France,"ArcelorMittal, ifp, Axens, Uetikon, Grassco, brevik, TotalEnergies (T&S)",Capture,2019.0,null,2025.0,null,Planned,2.0,0.7,0.7,Iron and steel,Unknown/unspecified,null,Europe,https://automotive.arcelormittal.com/news_and_stories/news/2019DMXproject,https://3d-ccus.com/3d-overview/,null,null,null,null,null,Link 1,Link 2,null,null,null,null,null
3D DMX ArcelorMittal and IFPEN Dunkirk 'REUZE',751,France,"ArcelorMittal, Engie, Infinium",CCU,2022.0,2024.0,2025.0,null,Planned,1.0,0.3,0.3,Iron and steel,Use,null,Europe,https://newsroom.engie.com/actualites/engie-et-infinium-annoncent-un-partenariat-pour-developper-un-hub-industriel-denvergure-europeenne-de-production-de-carburants-de-synthese-a-dunkerque-6d4d-ff316.html,https://corporate.arcelormittal.com/climate-action/decarbonisation-technologies/the-3d-project-dmx-demonstration-in-dunkirk,null,null,null,null,null,Link 1,Link 2,null,null,null,null,null
7 Blue Ammonia Facility,1055,Qatar,"QAFCO, thyssenkrupp Uhde/Consolidated Contractors Company (EPC)",Full Chain,2022.0,2022.0,2026.0,null,Under construction,null,1.5,1.5,Hydrogen or ammonia,Unknown/unspecified,null,Middle East,https://www.qafco.qa/content/qafco-7-announcement,https://www.thyssenkrupp.com/en/newsroom/press-releases/pressdetailpage/thyssenkrupp-uhde-to-build-world-scale-blue-ammonia-plant-in-qatar-138369,null,null,null,null,null,Link 1,Link 2,null,null,null,null,null


### Converter colunas em tipo inteiro
Quando fizemos a leitura do arquivo, algumas colunas foram inferidas como tipo float, quando na verdade tratam de valores inteiros.

In [0]:
colunas_cast_int = [
    "Announcement",
    "FID",
    "Operation",
    "Suspension_decommissioning",
    "Project_phase"
]
colunas_cast_int = {c: df[c].cast("int") for c in colunas_cast_int}

df = df.withColumns(colunas_cast_int)

display(df.limit(3))

Project_name,ID,Country,Partners,Project_type,Announcement,FID,Operation,Suspension_decommissioning,Project_Status,Project_phase,Announced_capacity_Mt_CO2_yr,Estimated_capacity_by_IEA_Mt_CO2_yr,Sector,Fate_of_carbon,Part_of_CCUS_hub,Region,Ref_1,Ref_2,Ref_3,Ref_4,Ref_5,Ref_6,Ref_7,Link_1,Link_2,Link_3,Link_4,Link_5,Link_6,Link_7
3D DMX ArcelorMittal and IFPEN Dunkirk (full-scale),1,France,"ArcelorMittal, ifp, Axens, Uetikon, Grassco, brevik, TotalEnergies (T&S)",Capture,2019,null,2025,null,Planned,2,0.7,0.7,Iron and steel,Unknown/unspecified,null,Europe,https://automotive.arcelormittal.com/news_and_stories/news/2019DMXproject,https://3d-ccus.com/3d-overview/,null,null,null,null,null,Link 1,Link 2,null,null,null,null,null
3D DMX ArcelorMittal and IFPEN Dunkirk 'REUZE',751,France,"ArcelorMittal, Engie, Infinium",CCU,2022,2024,2025,null,Planned,1,0.3,0.3,Iron and steel,Use,null,Europe,https://newsroom.engie.com/actualites/engie-et-infinium-annoncent-un-partenariat-pour-developper-un-hub-industriel-denvergure-europeenne-de-production-de-carburants-de-synthese-a-dunkerque-6d4d-ff316.html,https://corporate.arcelormittal.com/climate-action/decarbonisation-technologies/the-3d-project-dmx-demonstration-in-dunkirk,null,null,null,null,null,Link 1,Link 2,null,null,null,null,null
7 Blue Ammonia Facility,1055,Qatar,"QAFCO, thyssenkrupp Uhde/Consolidated Contractors Company (EPC)",Full Chain,2022,2022,2026,null,Under construction,null,1.5,1.5,Hydrogen or ammonia,Unknown/unspecified,null,Middle East,https://www.qafco.qa/content/qafco-7-announcement,https://www.thyssenkrupp.com/en/newsroom/press-releases/pressdetailpage/thyssenkrupp-uhde-to-build-world-scale-blue-ammonia-plant-in-qatar-138369,null,null,null,null,null,Link 1,Link 2,null,null,null,null,null


### Eliminar colunas Link
No arquivo Excel, as colunas Link possuem um hiperlink clicável para o endereço contido na coluna Ref de mesmo número. Esse recurso não se aplica a uma base de dados, por isso vamos remover as colunas Link e manter as colunas Ref.

In [0]:
colunas_drop = [c for c in df.columns if c.startswith("Link")]
df = df.drop(*colunas_drop)

display(df.limit(3))

Project_name,ID,Country,Partners,Project_type,Announcement,FID,Operation,Suspension_decommissioning,Project_Status,Project_phase,Announced_capacity_Mt_CO2_yr,Estimated_capacity_by_IEA_Mt_CO2_yr,Sector,Fate_of_carbon,Part_of_CCUS_hub,Region,Ref_1,Ref_2,Ref_3,Ref_4,Ref_5,Ref_6,Ref_7
3D DMX ArcelorMittal and IFPEN Dunkirk (full-scale),1,France,"ArcelorMittal, ifp, Axens, Uetikon, Grassco, brevik, TotalEnergies (T&S)",Capture,2019,null,2025,null,Planned,2,0.7,0.7,Iron and steel,Unknown/unspecified,null,Europe,https://automotive.arcelormittal.com/news_and_stories/news/2019DMXproject,https://3d-ccus.com/3d-overview/,null,null,null,null,null
3D DMX ArcelorMittal and IFPEN Dunkirk 'REUZE',751,France,"ArcelorMittal, Engie, Infinium",CCU,2022,2024,2025,null,Planned,1,0.3,0.3,Iron and steel,Use,null,Europe,https://newsroom.engie.com/actualites/engie-et-infinium-annoncent-un-partenariat-pour-developper-un-hub-industriel-denvergure-europeenne-de-production-de-carburants-de-synthese-a-dunkerque-6d4d-ff316.html,https://corporate.arcelormittal.com/climate-action/decarbonisation-technologies/the-3d-project-dmx-demonstration-in-dunkirk,null,null,null,null,null
7 Blue Ammonia Facility,1055,Qatar,"QAFCO, thyssenkrupp Uhde/Consolidated Contractors Company (EPC)",Full Chain,2022,2022,2026,null,Under construction,null,1.5,1.5,Hydrogen or ammonia,Unknown/unspecified,null,Middle East,https://www.qafco.qa/content/qafco-7-announcement,https://www.thyssenkrupp.com/en/newsroom/press-releases/pressdetailpage/thyssenkrupp-uhde-to-build-world-scale-blue-ammonia-plant-in-qatar-138369,null,null,null,null,null


### Tratar coluna Announced_capacity_Mt_CO2_yr
Essa coluna possui alguns valores expressos em faixas, como "0.3 - 0.8". Para permitir cálculos e agregações, vamos transformar essa coluna em duas: uma contendo o valor mínimo, outra contendo o valor máximo. Valores da coluna original que não sejam expressos em faixas serão repetidos, de forma a manter a integridade da informação de maneira independente nas duas colunas.

In [0]:
# Dividir a coluna
split_col = sf.split(df['Announced_capacity_Mt_CO2_yr'], "-")

# Criar coluna de valor mínimo
df = df.withColumn('Announced_capacity_low_Mt_CO2_yr', sf.when(sf.col('Announced_capacity_Mt_CO2_yr').contains('-'), split_col.getItem(0)).otherwise(sf.col('Announced_capacity_Mt_CO2_yr')).cast('double'))

# Criar coluna de valor máximo
df = df.withColumn('Announced_capacity_high_Mt_CO2_yr', sf.when(sf.col('Announced_capacity_Mt_CO2_yr').contains('-'), split_col.getItem(1)).otherwise(sf.col('Announced_capacity_Mt_CO2_yr')).cast('double'))

# Substituir coluna original pelas novas
colunas = df.columns
pos = colunas.index("Announced_capacity_Mt_CO2_yr")
colunas[pos] = "Announced_capacity_low_Mt_CO2_yr"
colunas.insert(pos+1, "Announced_capacity_high_Mt_CO2_yr")
colunas = colunas[:-2]
df = df.select(colunas)

display(df.limit(3))

Project_name,ID,Country,Partners,Project_type,Announcement,FID,Operation,Suspension_decommissioning,Project_Status,Project_phase,Announced_capacity_low_Mt_CO2_yr,Announced_capacity_high_Mt_CO2_yr,Estimated_capacity_by_IEA_Mt_CO2_yr,Sector,Fate_of_carbon,Part_of_CCUS_hub,Region,Ref_1,Ref_2,Ref_3,Ref_4,Ref_5,Ref_6,Ref_7
3D DMX ArcelorMittal and IFPEN Dunkirk (full-scale),1,France,"ArcelorMittal, ifp, Axens, Uetikon, Grassco, brevik, TotalEnergies (T&S)",Capture,2019,null,2025,null,Planned,2,0.7,0.7,0.7,Iron and steel,Unknown/unspecified,null,Europe,https://automotive.arcelormittal.com/news_and_stories/news/2019DMXproject,https://3d-ccus.com/3d-overview/,null,null,null,null,null
3D DMX ArcelorMittal and IFPEN Dunkirk 'REUZE',751,France,"ArcelorMittal, Engie, Infinium",CCU,2022,2024,2025,null,Planned,1,0.3,0.3,0.3,Iron and steel,Use,null,Europe,https://newsroom.engie.com/actualites/engie-et-infinium-annoncent-un-partenariat-pour-developper-un-hub-industriel-denvergure-europeenne-de-production-de-carburants-de-synthese-a-dunkerque-6d4d-ff316.html,https://corporate.arcelormittal.com/climate-action/decarbonisation-technologies/the-3d-project-dmx-demonstration-in-dunkirk,null,null,null,null,null
7 Blue Ammonia Facility,1055,Qatar,"QAFCO, thyssenkrupp Uhde/Consolidated Contractors Company (EPC)",Full Chain,2022,2022,2026,null,Under construction,null,1.5,1.5,1.5,Hydrogen or ammonia,Unknown/unspecified,null,Middle East,https://www.qafco.qa/content/qafco-7-announcement,https://www.thyssenkrupp.com/en/newsroom/press-releases/pressdetailpage/thyssenkrupp-uhde-to-build-world-scale-blue-ammonia-plant-in-qatar-138369,null,null,null,null,null


# Carga

## Definir descrição da tabela e comentários dos campos

In [0]:
descricao = "A tabela contém informações sobre projetos de captura, utilização e armazenamento de carbono (CCUS) ao redor do mundo. Ela inclui dados como o nome dos projetos, país, parceiros envolvidos, tipo de projeto, ano de anúncio, ano da decisão final de investimento, ano de operação, ano de suspensão/descomissionamento, status do projeto, fase do projeto, capacidade anunciada e estimada para captura, transporte e armazenamento de CO2, setor, destino do carbono, se o projeto faz parte de um hub CCUS, região, e links para artigos de notícias sobre o projeto. Esta tabela fornece insights valiosos sobre o panorama global dos projetos de CCUS e seu status atual."

In [0]:
comentarios = {
    "Project_name": "Nome do projeto",
    "ID": "Identificação única do projeto",
    "Country": "País onde o projeto está localizado",
    "Partners": "Parceiros envolvidos no projeto",
    "Project_type": "Tipo de projeto",
    "Announcement": "Ano de anúncio do projeto",
    "FID": "Ano da decisão final de investimento",
    "Operation": "Ano de início da operação do projeto",
    "Suspension_decommissioning": "Ano de suspensão ou descomissionamento do projeto",
    "Project_Status": "Status atual do projeto",
    "Project_phase": "Fase atual do projeto",
    "Announced_capacity_low_Mt_CO2_yr": "Capacidade mínima anunciada para captura de CO2 em milhões de toneladas por ano",
    "Announced_capacity_high_Mt_CO2_yr": "Capacidade máxima anunciada para captura de CO2 em milhões de toneladas por ano",
    "Estimated_capacity_by_IEA_Mt_CO2_yr": "Capacidade estimada pela IEA para captura de CO2 em milhões de toneladas por ano",
    "Sector": "Setor industrial do projeto",
    "Fate_of_carbon": "Destino do carbono capturado",
    "Part_of_CCUS_hub": "Indica se o projeto faz parte de um hub CCUS",
    "Region": "Região onde o projeto está localizado",
    "Ref_1": "Link para o primeiro artigo de notícias sobre o projeto",
    "Ref_2": "Link para o segundo artigo de notícias sobre o projeto",
    "Ref_3": "Link para o terceiro artigo de notícias sobre o projeto",
    "Ref_4": "Link para o quarto artigo de notícias sobre o projeto",
    "Ref_5": "Link para o quinto artigo de notícias sobre o projeto",
    "Ref_6": "Link para o sexto artigo de notícias sobre o projeto",
    "Ref_7": "Link para o sétimo artigo de notícias sobre o projeto"
}

In [0]:
# Notebook separado de análises